In [2]:
!pip install -q kaggle

In [13]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

!chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle datasets download -d muhammadhananasghar/human-emotions-datasethes


Dataset URL: https://www.kaggle.com/datasets/muhammadhananasghar/human-emotions-datasethes
License(s): unknown
 81% 252M/309M [00:00<00:00, 763MB/s] 
100% 309M/309M [00:01<00:00, 212MB/s]


In [15]:
!unzip "/content/human-emotions-datasethes.zip" -d "/content/dataset/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/EmotionsDataset/data/nothing/720.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/721.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/722.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/723.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/724.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/725.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/726.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/727.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/728.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/729.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/73.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/730.jpg  
  inflating: /content/dataset/EmotionsDataset/data/nothing/731.jpg  
  inflating: /content/dataset/EmotionsDataset/data/no

In [17]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import time
import random
from google.colab import files
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import matplotlib.cm as cm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers  import L2, L1
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature
from google.colab import drive

In [60]:
# from google.colab import files
# files.upload()

CONFIGURATION = {
    "BATCH_SIZE": 32,
    "IM_SIZE": 256,
    "LEARNING_RATE": 1e-3,
    "HIDDEN_DIM" : 768,
    "N_EPOCHS": 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASSES": 3,
    "PATCH_SIZE": 16,
    "PROJ_DIM": 768,
    "CLASS_NAMES": ["angry", "happy", "sad"],
}

train_dir = "/content/dataset/Emotions Dataset/Emotions Dataset/train"
val_dir = "/content/dataset/Emotions Dataset/Emotions Dataset/test"

In [8]:
!pip install wandb


In [9]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sayyamaggarwal2005 (sayyamaggarwal2005-guru-nanak-dev-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [10]:
import wandb
from wandb.integration.keras import WandbCallback

wandb.init(project='Emotion-Detection-Colab')

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: sayyamaggarwal2005 (sayyamaggarwal2005-guru-nanak-dev-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [18]:
wandb.config = CONFIGURATION

In [19]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode="rgb",
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99
)

Found 6799 files belonging to 3 classes.
Found 2278 files belonging to 3 classes.


In [21]:
class PatchEncodeLayer(Layer):
    def __init__(self,N_PATCHES,HIDDEN_SIZE):
        super(PatchEncodeLayer,self).__init__(name = "patch_encoder")
        self.LinearProj = Dense(HIDDEN_SIZE)
        self.postional_embed = tf.keras.layers.Embedding(N_PATCHES,HIDDEN_SIZE)
        self.n_patches = N_PATCHES

    def call(self,x):
        patches = tf.image.extract_patches(
            images=x,
            sizes=[1, CONFIGURATION['PATCH_SIZE'], CONFIGURATION['PATCH_SIZE'], 1],
            strides=[1, CONFIGURATION['PATCH_SIZE'],CONFIGURATION['PATCH_SIZE'], 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
            )

        patches = tf.reshape(patches,(tf.shape(patches)[0],256,patches.shape[-1]))
        embed_imput = tf.range(start = 0,limit = self.n_patches,delta =1)
        out = self.LinearProj(patches) + self.postional_embed(embed_imput)

        return out

In [22]:
patch_enc = PatchEncodeLayer(256,768)
patch_enc(tf.zeros([32,256,256,3]))

<tf.Tensor: shape=(32, 256, 768), dtype=float32, numpy=
array([[[-0.04254022, -0.03674461,  0.00505015, ..., -0.02696942,
          0.02636861, -0.00247927],
        [-0.03209485,  0.01864333, -0.02279344, ..., -0.03545939,
          0.03088934,  0.03668126],
        [ 0.00138953, -0.03503733, -0.01978268, ..., -0.00973486,
          0.02635891, -0.02830417],
        ...,
        [-0.01486016,  0.01913028, -0.04170684, ..., -0.00485406,
          0.01354435, -0.04937727],
        [-0.0053746 , -0.0351141 ,  0.008354  , ...,  0.03633923,
         -0.03666387, -0.04361282],
        [ 0.01146321,  0.00523861, -0.02113906, ...,  0.04308828,
         -0.0411927 ,  0.03039122]],

       [[-0.04254022, -0.03674461,  0.00505015, ..., -0.02696942,
          0.02636861, -0.00247927],
        [-0.03209485,  0.01864333, -0.02279344, ..., -0.03545939,
          0.03088934,  0.03668126],
        [ 0.00138953, -0.03503733, -0.01978268, ..., -0.00973486,
          0.02635891, -0.02830417],
        ...

In [23]:
# Transformation encoder

class TransformerEncoder(Layer):
    def __init__(self,N_HEADS,HIDDEN_SIZE):
        super(TransformerEncoder,self).__init__(name = 'transformer_encoder')
        # 1. Layer norm
        self.layer_norm1 = LayerNormalization()
        self.layer_norm2 = LayerNormalization()
        # 2. Multi head attention

        self.multiHead = tf.keras.layers.MultiHeadAttention(
            N_HEADS,HIDDEN_SIZE
        )
        # 3 dense layers

        self.dense_1 = Dense(HIDDEN_SIZE,activation = tf.nn.gelu)
        self.dense_2 = Dense(HIDDEN_SIZE,activation = tf.nn.gelu)


    def call(self,input):
        x = self.layer_norm1(input)
        x = self.multiHead(x,x)

        x = Add()([x,input])

        y = self.layer_norm2(x)


        y = self.dense_1(y)

        out = self.dense_2(y)

        output = Add()([out,x])

        return out


In [24]:
transformer_1 = TransformerEncoder(8,768)
transformer_1(tf.zeros([1,256,768]))

<tf.Tensor: shape=(1, 256, 768), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [25]:
#  Summing everything and making a a ViT

class ViT(Model):
    def __init__(self,N_HEADS,HIDDEN_SIZE,N_PATCHES,N_LAYERS,N_DENSE_UNITS):
        super(ViT,self).__init__(name = 'Vision_Transformer')
        self.n_layers = N_LAYERS
        self.patch_encoder = PatchEncodeLayer(N_PATCHES,HIDDEN_SIZE)
        self.transformer_encoder = [TransformerEncoder(N_HEADS,HIDDEN_SIZE) for _ in range(N_LAYERS)]
        self.dense_1 = Dense(N_DENSE_UNITS,activation=tf.nn.gelu)
        self.dense_2 = Dense(N_DENSE_UNITS,activation=tf.nn.gelu)
        self.dense_3 = Dense(CONFIGURATION['NUM_CLASSES'],activation='softmax')
    def call(self,input ,training=True):

        x = self.patch_encoder(input)
        for i in range(self.n_layers):
            x = self.transformer_encoder[i](x)

        x = Flatten()(x)

        x = self.dense_1(x)
        x = self.dense_2(x)
        return self.dense_3(x)



In [26]:
vit_model =  ViT(N_HEADS=4,HIDDEN_SIZE=768,N_PATCHES = 256,N_LAYERS=2,N_DENSE_UNITS=128)

vit_model(tf.zeros([32,256,256,3]))

<tf.Tensor: shape=(32, 3), dtype=float32, numpy=
array([[0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
       [0.38160995, 0.25433993, 0.36405015],
      

In [27]:
vit_model.summary()

Model: "Vision_Transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ patch_encoder                   │ ?                      │       787,200 │
│ (PatchEncodeLayer)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ ?                      │    10,631,424 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ ?                      │    10,631,424 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (32, 128)              │    25,165,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (32, 128)              │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (32, 3)                │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,232,899 (180.18 MB)

 Trainable params: 47,232,899 (180.18 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
metrics = [CategoricalAccuracy(name = "accuracy"), TopKCategoricalAccuracy(k=2, name = "top_k_accuracy")]
vit_model.compile(
    optimizer = Adam(learning_rate=CONFIGURATION['LEARNING_RATE']),
    loss = 'categorical_crossentropy',
    metrics = metrics
)
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,LearningRateScheduler,ModelCheckpoint,CSVLogger,TensorBoard
earlystop = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    restore_best_weights = True
)


reduced_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    min_lr=1e-6,
    verbose=1
)



In [29]:
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)


In [32]:
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
# callbacks = [
#     earlystop,
#     WandbMetricsLogger(log_freq="epoch"),
#     WandbModelCheckpoint(f"models/{run.id}")   # unique dir per run
# ]


In [36]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
# Login (if not already done)

# Initialize the run
run = wandb.init(project="Emotion-Detection-Colab")

# Define callbacks
checkpoint_dir = f"models/{run.id}"
os.makedirs(checkpoint_dir, exist_ok=True)

callbacks = [
    earlystop,
    WandbMetricsLogger(log_freq="epoch"),
    WandbModelCheckpoint(f"{checkpoint_dir}/model.keras")  # filename required
]

history = vit_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=CONFIGURATION['N_EPOCHS'],
    verbose=1,
    callbacks=callbacks
)


Epoch 1/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 119s 559ms/step - accuracy: 0.4283 - loss: 1.1079 - top_k_accuracy: 0.7468 - val_accuracy: 0.4416 - val_loss: 1.0747 - val_top_k_accuracy: 0.7739
Epoch 2/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 117s 550ms/step - accuracy: 0.4538 - loss: 1.0652 - top_k_accuracy: 0.7627 - val_accuracy: 0.4631 - val_loss: 1.0526 - val_top_k_accuracy: 0.7735
Epoch 3/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 111s 523ms/step - accuracy: 0.4446 - loss: 1.0625 - top_k_accuracy: 0.7643 - val_accuracy: 0.4469 - val_loss: 1.0728 - val_top_k_accuracy: 0.7682
Epoch 4/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 119s 557ms/step - accuracy: 0.4568 - loss: 1.0643 - top_k_accuracy: 0.7708 - val_accuracy: 0.4552 - val_loss: 1.0535 - val_top_k_accuracy: 0.7722
Epoch 5/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 118s 553ms/step - accuracy: 0.4708 - loss: 1.0531 - top_k_accuracy: 0.7701 - val_accuracy: 0.4846 - val_loss: 1.0415 - val_top_k_accuracy: 0.7717
Epoch 6/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 133s 509ms/step - accuracy: 0.

In [38]:
# Finetuning a huggiung face transformer

!pip install transformers

In [39]:
resize_rescale_hf = tf.keras.Sequential([
       Resizing(224, 224),
       Rescaling(1./255),
       Permute((3,1,2))
])

In [68]:
from transformers import ViTFeatureExtractor, TFViTModel
from tensorflow.keras.layers import Lambda

base_model = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k",from_pt = True)

inputs = Input(shape = (CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"],3),dtype = tf.float32)

# Resize, rescale, and permute dimensions to match the expected input format
x = resize_rescale_hf(inputs)

# Pass through TFViTModel using Lambda to convert KerasTensor to tensor
cls_token = Lambda(lambda img: base_model(pixel_values=img).last_hidden_state[:,0,:])(x)

# Add classification head
x = Dense(256, activation='relu')(cls_token)
outputs = Dense(CONFIGURATION['NUM_CLASSES'], activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [69]:
test_image = cv2.imread("/content/dataset/Emotions Dataset/Emotions Dataset/train/happy/387249.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))

In [70]:
model(tf.expand_dims(test_image,axis = 0))

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.4235495 , 0.28561142, 0.29083908]], dtype=float32)>

In [71]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 3, 224, 224)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_8 (Lambda)               │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 197,635 (772.01 KB)

 Trainable params: 197,635 (772.01 KB)

 Non-trainable params: 0 (0.00 B)

In [1]:
run_new = wandb.init(project='Emotion-Detection-Colab-GoogleTransformers')
checkpoint_dir_new = f"models/{run_new.id}"

os.makedirs(checkpoint_dir_new, exist_ok=True)

callbacks_new = [
    earlystop,
    WandbMetricsLogger(log_freq="epoch"),
    WandbModelCheckpoint(f"{checkpoint_dir_new}/model.keras")  # filename required
]

model.compile(
    optimizer = Adam(learning_rate=5e-5),#low learning rate for google transoformers
    loss = 'categorical_crossentropy',
    metrics = metrics
)

history_new = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=CONFIGURATION['N_EPOCHS'],
    verbose=1,
    callbacks=callbacks_new
)

NameError: name 'wandb' is not defined

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@87c14b1dfd77.(none)')


Streaming output truncated to the last 5000 lines.
 create mode 100644 dataset/EmotionsDataset/data/nothing/778.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/779.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/78.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/780.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/781.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/782.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/783.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/784.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/785.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/786.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/787.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/788.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/789.jpg
 create mode 100644 dataset/EmotionsDataset/data/nothing/79.jpg
 create mode 100644 dataset/EmotionsDatas

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> EmotionsPred
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached EmotionsPred
hint: 
hint: See "git help submodule" for more information.


[main 9293e43] trained transformers in google colab gpu
 3 files changed, 17096 insertions(+)
 create mode 160000 EmotionsPred


/content/EmotionsPred


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Generating public/private ed25519 key pair.
/root/.ssh/id_ed25519 already exists.
Overwrite (y/n)? y
Your identification has been saved in /root/.ssh/id_ed25519
Your public key has been saved in /root/.ssh/id_ed25519.pub
The key fingerprint is:
SHA256:Q3sXRvqGmo81z5HNIfAIlJ0Vb6N54LruMxEy7C4D3yU sayyamaggarwal2005@gmail.com
The key's randomart image is:
+--[ED25519 256]--+
|        .o o+.   |
|       .. oo .   |
|       .o o + +  |
|       .+o.O * . |
|       .So+.X o  |
|    .   E=o+ * . |
|     o ooo+.o o  |
|      + o+o= .   |
|       o.o=oo    |
+----[SHA256]-----+
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIKY/zIYH1ktDq4S98kv3+WJQv/jN14sI4jjFJnPVcWZU sayyamaggarwal2005@gmail.com


Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


id_ed25519  id_ed25519.pub


ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIKY/zIYH1ktDq4S98kv3+WJQv/jN14sI4jjFJnPVcWZU sayyamaggarwal2005@gmail.com


# github.com:22 SSH-2.0-88f9acd9
# github.com:22 SSH-2.0-88f9acd9
# github.com:22 SSH-2.0-88f9acd9
# github.com:22 SSH-2.0-88f9acd9
# github.com:22 SSH-2.0-88f9acd9


github.com ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQCj7ndNxQowgcQnjshcLrqPEiiphnt+VTTvDP6mHBL9j1aNUkY4Ue1gvwnGLVlOhGeYrnZaMgRK6+PKCUXaDbC7qtbW8gIkhL7aGCsOr/C56SJMy/BCZfxd1nWzAOxSDPgVsmerOBYfNqltV9/hWCqBywINIR+5dIg6JTJ72pcEpEjcYgXkE2YEFXV1JHnsKgbLWNlhScqb2UmyRkQyytRLtL+38TGxkxCflmO+5Z8CSSNY7GidjMIZ7Q4zMjA2n1nGrlTDkzwDCsw+wqFPGQA179cnfGWOWRVruj16z6XyvxvjJwbz0wQZ75XK5tKSb7FNyeIEs4TT4jk+S4dhPeAUC5y+bDYirYgM4GC7uEnztnZyaVWQ7B381AK4Qdrwt51ZqExKbQpTUNn+EjqoTwvqNj4kqx5QUCI0ThS/YkOxJCXmPUWZbhjpCg56i+2aB6CmK2JGhn57K5mj0MNdBXA4/WnwH6XoPWJzK5Nyu2zB3nAZp+S5hpQs+p1vN1/wsjk=
github.com ecdsa-sha2-nistp256 AAAAE2VjZHNhLXNoYTItbmlzdHAyNTYAAAAIbmlzdHAyNTYAAABBBEmKSENjQEezOmxkZMy7opKgwFB9nkt5YRrYMjNuG5N87uRgg6CLrbo5wAdT/y6v0mKV0U2w0WZ2YB/++Tpockg=
github.com ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIOMqqnkVzrm0SdG6UOoqKLsabgH5C9okWi0dh2l9GKJl


Hi SayyamAggarwal! You've successfully authenticated, but GitHub does not provide shell access.


/content/EmotionsPred
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Branch 'main' set up to track remote branch 'main' from 'origin'.
Everything up-to-date


env_path.txt		 models		       resnet34_history.pkl
kaggle.json		 requirements.txt      ViT.ipynb
Lenet_model_build.ipynb  ResNet34_Build.ipynb  wandb


dataset       human-emotions-datasethes.zip  models	  wandb
EmotionsPred  kaggle.json		     sample_data


Mounted at /content/drive


/content/drive/MyDrive/Colab Notebooks/training.ipynb


Saving training.ipynb to training.ipynb


/content/EmotionsPred
[main 741fd64] Remove kaggle.json for security
 1 file changed, 1 deletion(-)
 delete mode 100644 kaggle.json
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (2/2), 250 bytes | 250.00 KiB/s, done.
Total 2 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:SayyamAggarwal/EmotionsPred.git
   8bc67ff..741fd64  main -> main
[main 523e89f] Add kaggle.json to .gitignore
 1 file changed, 1 insertion(+)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 320 bytes | 320.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:SayyamAggarwal/EmotionsPred.git
   741fd6

[main ec31f74] Add files from Colab
 56 files changed, 38632 insertions(+)
 create mode 100644 models/i7pw3lar/model.keras
 create mode 100644 models/kkomvqgv/model.keras
 create mode 100644 training.ipynb
 create mode 120000 wandb/debug-internal.log
 create mode 120000 wandb/debug.log
 create mode 120000 wandb/latest-run
 create mode 100644 wandb/run-20250906_190159-x6ebinb7/files/config.yaml
 create mode 100644 wandb/run-20250906_190159-x6ebinb7/files/media/graph/graph_summary_ceacf82e40e0d41306a6.graph.json
 create mode 100644 wandb/run-20250906_190159-x6ebinb7/files/model-best.h5
 create mode 100644 wandb/run-20250906_190159-x6ebinb7/files/output.log
 create mode 100644 wandb/run-20250906_190159-x6ebinb7/files/requirements.txt
 create mode 100644 wandb/run-20250906_190159-x6ebinb7/files/wandb-metadata.json
 create mode 100644 wandb/run-20250906_190159-x6ebinb7/files/wandb-summary.json
 create mode 120000 wandb/run-20250906_190159-x6ebinb7/logs/debug-core.log
 create mode 100644 wan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 kB 4.8 MB/s eta 0:00:00
Aborting: Refusing to destructively overwrite repo history since
this does not look like a fresh clone.
  (expected freshly packed repo)
Please operate on a fresh clone instead.  If you want to proceed
anyway, use --force.
Aborting: Refusing to destructively overwrite repo history since
this does not look like a fresh clone.
  (expected freshly packed repo)
Please operate on a fresh clone instead.  If you want to proceed
anyway, use --force.
Enumerating objects: 79, done.
Counting objects: 100% (79/79), done.
Delta compression using up to 2 threads
Compressing objects: 100% (65/65), done.
Writing objects: 100% (77/77), 475.73 MiB | 16.07 MiB/s, done.
Total 77 (delta 21), reused 1 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (21/21), completed with 1 local object.
remote: error: Trace: 18ff83c0ab1a03cae7cd7c7fdb8d3f1b9d5714d4caade939631d35ebf25d6fee
remote: error: See https://gh.io/lfs for more info